# 微调GPT2生成正面评论


> 将BERT情感分类器作为奖励模型(RM, reward model)来优化GPT2生成正面的额IMDB电影评论。

典型的RLHF微调模式：
- SFT：GPT2模型，这里没有做进一步的有监督微调；
- RM：使用BERT情感分类模型作为奖励模型
- RL：使用PPO进行强化学习训练，引导模型生成

鼎鼎大名的ChatGPT的训练三步法，如下如所示。
![](https://pic3.zhimg.com/80/v2-85fc3194aa6e8c7202048a9143da998a_1440w.webp)

<div style="text-align: center">
<img src='https://huggingface.co/datasets/trl-internal-testing/example-images/resolve/main/images/gpt2_bert_training.png' width='600'>
<p style="text-align: center;"> <b>Figure:</b> 微调GPT2的实验流程. </p>
</div>

>本文微调一个GPT2（相比ChatGPT是非常小了），来生成正面的电影评论。给模型输入一个电影评论，然后生成一个正面的较长的评论。使用BERT分类器来评价生成评论的正面程度，强化学习算法是ChatGPT同款PPO（CloseAI自己提出来的）。

## 基本设置

### 安装和加载依赖包

In [ ]:
# %load_ext autoreload
# %autoreload 2
!pip install git+https://github.com/huggingface/transformers.git -U
!pip install -U accelerate
!pip install git+https://github.com/huggingface/peft.git -U
%pip install trl wandb bitsandbytes loralib sentencepiece appdirs

In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM

In [ ]:
import torch
from tqdm import tqdm
import pandas as pd
tqdm.pandas()
from datasets import load_dataset

# 使用lora进行微调，速度更快，效果和直接微调原模型效果接近（通过实验验证）。
from peft import LoraConfig, PeftConfig, PeftModel, get_peft_model, prepare_model_for_int8_training

# trl包，是transformer reinforcement learning的首字母缩写，提供了非常方便的transformer模型的RLHF训练API
# 感谢开源！有兴趣的可以去github上的
from trl import PPOTrainer, PPOConfig, AutoModelForCausalLMWithValueHead
from trl.core import LengthSampler

### 配置

In [ ]:
config = PPOConfig(
    model_name="lvwerra/gpt2-imdb", # 这个是Huggingface上要训练的gpt2-imdb的名称，在transformer中可以用from_pretrained直接下载和缓存
    learning_rate=1.41e-5, # 学习率
#     log_with="wandb", # 使用wandb监视训练过程，也可以使用tensorboard
#     log_with="tensorboard", # 使用wandb监视训练过程，也可以使用tensorboard
#     accelerator_kwargs={"logging_dir": "./tb_logger"}
)

sent_kwargs = {
    "return_all_scores": True, # 文本生成的参数，这里设置为True，表示生成文本时返回得分
    "function_to_apply": "none", 
    "batch_size": 16 # 批大小，不解释了。玩深度学习这个读懂。GPU显存越大，这个可以设的越大。
}

```python
# 使用transformers的AutoModelForXXX来加载模型
pretrained_model = AutoModelForCausalLM.from_pretrained(config.model_name, load_in_8bit=True, device_map="auto")

# 设置目标模块名称
target_modules = None
if "gpt-neox" in script_args.model_name:
    target_modules = ["query_key_value", "xxx"]  # workaround to use 8bit training on this model

# 设置lora配置参数
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=target_modules,  # handled automatically by peft
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

# 设置8bit训练
pretrained_model = prepare_model_for_int8_training(pretrained_model, output_embedding_layer_name="embed_out")

# 设置lora模型
pretrained_model = get_peft_model(pretrained_model, lora_config)

# 将lora模型加载入trl模型，加上value head
model = AutoModelForCausalLMWithValueHead.from_pretrained(pretrained_model)

# 做必要的设置，梯度检查。
model.gradient_checkpointing_disable = model.pretrained_model.gradient_checkpointing_disable
model.gradient_checkpointing_enable = model.pretrained_model.gradient_checkpointing_enable
```

In [ ]:
# step 1: 使用transformers库加载模型
pretrained_model = AutoModelForCausalLM.from_pretrained(
    config.model_name,
#     device_map="auto",
#     load_in_8bit=True,
)

# 设置目标模块名称
target_modules = None
target_modules = ["c_attn"]  # workaround to use 8bit training on this model

# 设置lora配置参数
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=target_modules,  # handled automatically by peft
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

# step 2: 设置8bit训练
pretrained_model = prepare_model_for_int8_training(pretrained_model, output_embedding_layer_name="lm_head")
# for name, param in pretrained_model.named_parameters():
#     # freeze base model's layers
#     param.requires_grad = False

#     if getattr(pretrained_model, "is_loaded_in_8bit", False):
#         # cast layer norm in fp32 for stability for 8bit models
#         if param.ndim == 1 and "layer_norm" in name:
#             param.data = param.data.to(torch.float16)

# step 3: 设置lora模型。做instruction learning，到这里就好了。如果要做RLHF，还要做第四步。
pretrained_model = get_peft_model(pretrained_model, lora_config)

# step 4: 将lora模型加载入trl模型，加上value head。
model = AutoModelForCausalLMWithValueHead.from_pretrained(pretrained_model)

# 做必要的设置，梯度检查。
model.gradient_checkpointing_disable = model.pretrained_model.gradient_checkpointing_disable
model.gradient_checkpointing_enable = model.pretrained_model.gradient_checkpointing_enable

# model.gradient_checkpointing_disable()
# model.pretrained_model.config.use_cache = True
# ppo_trainer = PPOTrainer(config, model, ref_model=None, 
#                          tokenizer=tokenizer, dataset=dataset, 
#                          data_collator=collator)
# ppo_trainer.generate(query, **generation_kwargs)

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )
    
print_trainable_parameters(model)

In [ ]:
# import wandb
# from kaggle_secrets import UserSecretsClient
# user_secrets = UserSecretsClient()
# wandb_api = user_secrets.get_secret("wandb_key")
# wandb.login(key=wandb_api)
# wandb.init(project="trl_imdb_positive")

> 可以看到前面设置了`gpt2_imdb`这样一个GPT2模型。这个模型已经在IMDB数据集上进行了微调，也就是SFT步骤。具体微调模型的代码可以在链接[script](https://github.com/huggingface/transformers/blob/master/examples/run_language_modeling.py)找到。具体论文可以参考链接["Fine-Tuning Language Models from Human Preferences"](https://arxiv.org/pdf/1909.08593.pdf)。在Huggingface的Hub上可以找到GPT2和BERT
对应的IMDB微调模型，链接为[here](https://huggingface.co/models)。下面的代码在kaggle上运行，会自动加载缓存数据集和模型。

## 加载数据和模型

### 加载IMDB数据集

> IMDB数据集包含50,000个电影评论，已经用`positive/negative`标注好了情感极性。我们将IMDB数据集转换为Pandas的DataFrame(可以参考pandas的文档)，并过滤掉评论超过200个字符的评论。然后对文本进行分词，并使用`LengthSampler`来将其切割为随机尺寸（保证模型对不同长度的评论都有作用）。

In [ ]:
def build_dataset(config, dataset_name="imdb", input_min_text_length=2, input_max_text_length=8):
    """
    构建训练用的数据集。使用`load_dataset`函数下载和缓存数据集。如果要用自己的数据集，则需要替换该部分代码。
    当然`load_dataset`也可以加载本地数据集，详情各自行百度，或者去datasets的官网查找帮助信息。
    
    Args:
        dataset_name (`str`): 
            数据集名称
    
    Returns:
        dataloader (`torch.utils.data.DataLoader`):
            返回dataloader
    """
    tokenizer = AutoTokenizer.from_pretrained(config.model_name)
    tokenizer.pad_token = tokenizer.eos_token # pad_token和eos_token是同一个，也可以用其它的token进行替换。
    # 加载IMDB数据集，直接从huggingface的hub上下载数据，当然也可以下载其他数据
    # 每次做DL或ML时，大量时间用在了做
    ds = load_dataset(dataset_name, split='train') # 加载后是DataFrame格式！？
    ds = ds.rename_columns({'text': 'review'})
    ds = ds.filter(lambda x: len(x["review"])>200, batched=False) # 这里filter是指len(x["review"])>200都过滤掉

    # 对batch_size进行裁剪，缩小到2到8之间。（2和8是函数中的默认参数）
    # 即query的token长度控制在2到8之间，有点小呀
    input_size = LengthSampler(input_min_text_length, input_max_text_length)
    
    def tokenize(sample):
        sample["input_ids"] = tokenizer.encode(sample["review"])[:input_size()] # 后面设置batched=False,每次input_size都不同
        sample["query"] = tokenizer.decode(sample["input_ids"])
        return sample

    ds = ds.map(tokenize, batched=False)
    # 将数值型变量设置为torch的tensor格式，并且输出所有的列数据，在RL截断需要使用！一定要注意设置output_all_columns=True
    ds.set_format(type='torch', columns=["input_ids", "label"], output_all_columns=True)
    return ds

In [ ]:
dataset = build_dataset(config)

def collator(data):
    return dict((key, [d[key] for d in data]) for key in data[0])

### 加载已经预训练好的GPT2语言模型

> 这里加载带有value head的GPT2模型及其对应的分词器。下面加载了两次模型；第一次加载的模型用来进行强化学习，调整参数。第二次加载的模型作为参考模型，用来计算和前面可训练模型的KL散度。这个KL散度，用来作为PPO训练的额外奖励信号，来保证我们的模型不会太偏离原始模型（即防止灾难性遗忘情况的发生）。

In [ ]:
# lora_config = LoraConfig(
#     r=16,
#     lora_alpha=32,
#     lora_dropout=0.05,
#     bias="none",
#     task_type="CAUSAL_LM",
# )

# 下面的代码会报错！
# model = AutoModelForCausalLMWithValueHead.from_pretrained(
#     config.model_name,
#     device_map="auto",
#     load_in_8bit=True,
#     peft_config=lora_config,
#     layer_norm_names=[]
# )

In [ ]:
ref_model = AutoModelForCausalLMWithValueHead.from_pretrained(config.model_name)
tokenizer = AutoTokenizer.from_pretrained(config.model_name)

tokenizer.pad_token = tokenizer.eos_token

In [ ]:
ppo_trainer = PPOTrainer(config, model, ref_model=ref_model, 
                         tokenizer=tokenizer, dataset=dataset, 
                         data_collator=collator)

### 初始化PPOTrainer

> `PPOTrainer`可以很方便的进行训练，并且处理好数据在GPU上的计算。

### 加载BERT分类器
> 下面加载在IMDB数据集上微调过的BERT分类器

In [ ]:
device = ppo_trainer.accelerator.device
if ppo_trainer.accelerator.num_processes == 1:
    device = 0 if torch.cuda.is_available() else "cpu" # to avoid a `pipeline` bug
sentiment_pipe = pipeline("sentiment-analysis", model="lvwerra/distilbert-imdb", device=device)

> 模型会输出数值，来对应正面和负面的类别。下面使用正面类的得分作为语言模型的奖励信号。

In [ ]:
# text = 'this movie was really bad!!'
# sentiment_pipe(text, **sent_kwargs)

In [ ]:
# text = 'this movie was really good!!'
# pipe_outputs = sentiment_pipe([text, text], **sent_kwargs)

In [ ]:
# output = pipe_outputs[0]
# print(output)
# print()
# print(output[1])

> 通过下面的代码来获取正面得分

In [ ]:
# rewards = [torch.tensor(output[1]["score"]) for output in pipe_outputs]
# rewards

### 文本生成设置

> 根据query生成response，这里的配置使用top_p和随机采样来生成文本。

In [ ]:
gen_kwargs = {
    "min_length":-1,
    "top_k": 0.0,
    "top_p": 1.0,
    "do_sample": True,
    "pad_token_id": tokenizer.eos_token_id
}

## 使用PPO强化学习算法优化模型

### 训练循环

训练循环主要包含三个步骤：
- 根据query，基于GPT2生成response
- 拼接query和response，使用BERT来得到拼接后文本的得分
- 基于(query, response, reward)三元组，基于PPO算法来优化模型

**训练耗时**

基于上述配置，在V100上大约耗时两个小时完成训练。（如果使用peft包，可能会快一些，但是效果不知道怎么样！）

In [ ]:
output_min_length = 4
output_max_length = 16
output_length_sampler = LengthSampler(output_min_length, output_max_length)


generation_kwargs = {
    "min_length":-1,
    "top_k": 0.0,
    "top_p": 1.0,
    "do_sample": True,
    "pad_token_id": tokenizer.eos_token_id
}


for epoch, batch in tqdm(enumerate(ppo_trainer.dataloader)):
    query_tensors = batch['input_ids']
    
    model.gradient_checkpointing_disable()
    model.pretrained_model.config.use_cache = True
    #### Get response from gpt2
    response_tensors = []
    for query in query_tensors:
        gen_len = output_length_sampler()
        generation_kwargs["max_new_tokens"] = gen_len
        response = ppo_trainer.generate(query, **generation_kwargs)
        response_tensors.append(response.squeeze()[-gen_len:])
    batch['response'] = [tokenizer.decode(r.squeeze()) for r in response_tensors]

    #### Compute sentiment score
    texts = [q + r for q,r in zip(batch['query'], batch['response'])]
    pipe_outputs = sentiment_pipe(texts, **sent_kwargs)
    rewards = [torch.tensor(output[1]["score"]) for output in pipe_outputs]
    
    # Run PPO step
    model.gradient_checkpointing_enable()
    model.pretrained_model.config.use_cache = False
    
    #### Run PPO step 
    stats = ppo_trainer.step(query_tensors, response_tensors, rewards)
    ppo_trainer.log_stats(stats, batch, rewards)
    
#     break

### 训练过程
如果使用Weights&Biases来跟踪训练过程，可以查看[链接](https://app.wandb.ai/lvwerra/trl-showcase/runs/1jtvxb1m/).该链接是已经训练好wandb工程。

<div style="text-align: center">
<img src='https://huggingface.co/datasets/trl-internal-testing/example-images/resolve/main/images/gpt2_tuning_progress.png' width='800'>
<p style="text-align: center;"> <b>图1:</b> 训练过程中的奖励均值和分布变化情况</p>
</div>

在训练一段时间之后，可以观察到该模型会生成正面的评论文本，表明模型的训练趋势是正确的。

## 测试模型效果
让我们测试几个IMDB的评论生成例子。比较`model`和`model_ref`的生成结果和评论得分。

In [ ]:
#### get a batch from the dataset
bs = 16
game_data = dict()
dataset.set_format("pandas")
df_batch = dataset[:].sample(bs)
game_data['query'] = df_batch['query'].tolist()
query_tensors = df_batch['input_ids'].tolist()

response_tensors_ref, response_tensors = [], []

#### get response from gpt2 and gpt2_ref
for i in range(bs):
    gen_len = output_length_sampler()
    output = ref_model.generate(torch.tensor(query_tensors[i]).unsqueeze(dim=0).to(device),
                                     max_new_tokens=gen_len, **gen_kwargs).squeeze()[-gen_len:]
    response_tensors_ref.append(output) # 
#     output = model.generate(torch.tensor(query_tensors[i]).unsqueeze(dim=0).to(device),
#                                  max_new_tokens=gen_len, **gen_kwargs).squeeze()[-gen_len:]
    output = ppo_trainer.generate(torch.tensor(query_tensors[i]).to(device),
                                 max_new_tokens=gen_len, **gen_kwargs).squeeze()[-gen_len:]
    response_tensors.append(output)

#### decode responses
game_data['response (before)'] = [tokenizer.decode(response_tensors_ref[i]) for i in range(bs)]
game_data['response (after)'] = [tokenizer.decode(response_tensors[i]) for i in range(bs)]

#### sentiment analysis of query/response pairs before/after
texts = [q + r for q,r in zip(game_data['query'], game_data['response (before)'])]
game_data['rewards (before)'] = [output[1]["score"] for output in sentiment_pipe(texts, **sent_kwargs)]

texts = [q + r for q,r in zip(game_data['query'], game_data['response (after)'])]
game_data['rewards (after)'] = [output[1]["score"] for output in sentiment_pipe(texts, **sent_kwargs)]

# store results in a dataframe
df_results = pd.DataFrame(game_data)
df_results

然后观察下生成序列的奖励均值和中位数，存在明显的区别，如下。

In [ ]:
print('mean:')
display(df_results[["rewards (before)", "rewards (after)"]].mean())
print()
print('median:')
display(df_results[["rewards (before)", "rewards (after)"]].median())

## 保存模型
最后，将模型保存到HuggingFace的官网上。

In [ ]:
# 登录huggingface Hub
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
huggingface_key = user_secrets.get_secret("huggingface_key")
login(token=huggingface_key, add_to_git_credential=True)

In [ ]:
model.save_pretrained('gpt2-imdb-pos-v2', push_to_hub=True)
tokenizer.save_pretrained('gpt2-imdb-pos-v2', push_to_hub=True)